# Многослойный перцептрон

## Лабораторная работа №4

Присвойте строку с вашим ФИО переменной `name` и выполните ячейку. В ее выводе будет указан номер вашего варианта.

In [1]:
name = "Сильченко Алексей Евгеньевич" # Впишите ваше ФИО

def calculate_variant(name):
    return sum(ord(char) for char in name) % 2 + 1

print(f"Ваш вариант - №{calculate_variant(name)}")

Ваш вариант - №1



---

**Впишите в эту ячейку ваши ФИО, группу и вариант**.

ФИО: Сильченко Алексей Евгеньевич

Группа: 201-361

Вариант: 1

---

Далее по ходу ноутбука вам встрется ячейки с кодом, в которых будут комментарии с заданиями, и текстовые ячейки как эта с вопросами, на которые вам необходимо письменно в ноутбуке ответить.

Все ячейки необходимо запускать.

Данные для вариантов лежат по ссылкам:

1. http://labcolor.space/mlp-1.csv
1. http://labcolor.space/mlp-2.csv

Скопируйте ссылку для своего варианта.

## Импорт модулей

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Загрузка данных

Используя уже знакомые вам функции, загрузите данные и проведите их анализ на предмет количества признаков, объектов и классов. В данных последний столбец это целевой признак, а первая строка — это порядковый номер признака начиная с 0.

In [3]:
#
# Ваш код

# Загрузка данных
url = "http://labcolor.space/mlp-1.csv"
data = pd.read_csv(url)

# Анализ данных
number_of_features = data.shape[1] - 1  # Вычитаем 1, так как последний столбец - это целевой признак
number_of_objects = data.shape[0]
unique_classes = data.iloc[:, -1].nunique()

# Вывод результатов
print("Количество признаков:", number_of_features)
print("Количество объектов:", number_of_objects)
print("Количество уникальных классов:", unique_classes)

Количество признаков: 100
Количество объектов: 3000
Количество уникальных классов: 6


**Описание данных**

Ваш ответ: 
Количество признаков: 100
Количество объектов: 3000
Количество уникальных классов: 6

## Подготовка данных

Убедитесь что в данных нет отсутствующих значений, если они есть, удалите их или заполните средними значениями - `df.isnull()`.

In [4]:
# Ваш код

if data.isnull().values.any():
    print("В данных есть отсутствующие значения.")
else:
    print("Отсутствующих значений в данных нет.")


missing_values = data.isnull().sum()
print("Отсутствующие значения в каждом столбце:\n", missing_values)

# Удаление строк с отсутствующими значениями (или можно заменить средними значениями)
data = data.dropna()

# Проверка после обработки
if data.isnull().values.any():
    print("В данных есть отсутствующие значения.")
else:
    print("Отсутствующих значений в данных нет.")

В данных есть отсутствующие значения.
Отсутствующие значения в каждом столбце:
 0      0
1      0
2      0
3      0
4      0
      ..
96     0
97     0
98     0
99     1
100    0
Length: 101, dtype: int64
Отсутствующих значений в данных нет.


**Были ли отсутствующие значения и что вы с ними сделали?**

Ваш ответ: Да, удалил т.к. их было не много

С помощью `train_test_split` разделите данные на обучающие и тестовые (15-20%). Не забудьте про `random_state`.

In [5]:
from sklearn.model_selection import train_test_split

# Определение признаков (X) и целевой переменной (y)
X = data.iloc[:, :-1]  # Все столбцы, кроме последнего
y = data.iloc[:, -1]   # Последний столбец

# Разделение данных на обучающие и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)  

# Вывод размеров полученных наборов данных
print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)

Размер обучающего набора: (2537, 100)
Размер тестового набора: (448, 100)


Масштабируйте признаки. Для этого импортируйте соответствующий класс, создайте его экземпляр, выполните его "подгонку" и примените его к признакам.

In [6]:
from sklearn.preprocessing import StandardScaler

# Создание экземпляра StandardScaler
scaler = StandardScaler()

# "Подгонка" масштабировщика на обучающих данных и применение его к обучающему набору
X_train_scaled = scaler.fit_transform(X_train)

# Применение масштабировщика к тестовому набору (используя те же параметры, что и для обучающего набора)
X_test_scaled = scaler.transform(X_test)


## Обучение модели и подборка гиперпараметров

Гиперпараметры и их значения задаются в виде словаря и затем вместе с экземпляром модели в качестве аргументов передаются в конструктор класса `GridSearchCV`, который будет создавать разные варианты модели и обучать их, используя кросс валидацию, сохраняя промежуточные результаты.

Попробуйте разные варианты:
* количества нейронов в скрытых слоях (hidden_layer_sizes),
* функций активации (activation),
* скорости обучения (learning_rate_init).

**Что такое гиперпараметры для модели?**

Ваш ответ: Гиперпараметры – это параметры, которые задаются до начала обучения модели и не изменяются в процессе обучения.

**Опишите суть кросс валидации.**

Ваш ответ: Кросс-валидация – это метод оценки обобщающей способности модели, который позволяет более надёжно оценить её работу на независимых данных. 

**В чем заключается идея поиска по сетке (grid search)?**

Ваш ответ: это метод подбора оптимальных гиперпараметров для модели.

In [7]:

# Задание сетки гиперпараметров для перебора
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Разные комбинации нейронов в скрытых слоях
    'activation': ['tanh', 'relu'],                              # Функции активации
    'learning_rate_init': [0.001, 0.01]                          # Скорости обучения
}


**Сколько вариантов модели будет в итоге обучено?**

Ваш ответ: 4 * 2 * 2 = 16

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [9]:
mlp = MLPClassifier(max_iter=1000, random_state=42)

grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train) # Запустите обучение с нужными данными

GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=1000, random_state=42),
             param_grid={'activation': ['tanh', 'relu'],
                         'hidden_layer_sizes': [(50,), (100,), (50, 50),
                                                (100, 50)],
                         'learning_rate_init': [0.001, 0.01]},
             scoring='accuracy')

**Что значит cv=3 для GridSearchCV?**

Ваш ответ:
Параметр cv=3 в GridSearchCV означает, что для оценки каждой комбинации гиперпараметров используется 3-кратная кросс-валидация. Это означает следующее:

Весь набор данных разделяется на 3 части (или "фолды").
Для каждой комбинации гиперпараметров модель обучается и тестируется 3 раза:
- В первой итерации первый фолд используется в качестве тестового набора, а остальные два - как обучающие.
- Во второй итерации второй фолд становится тестовым, а первый и третий - обучающими.
- В третьей итерации третий фолд используется для тестирования, а первые два - для обучения.
Результаты каждого из этих тестов усредняются, чтобы получить оценку эффективности данной комбинации гиперпараметров.

In [10]:
print(f"Гиперпараметры модели с лучшим результатом: {grid_search.best_params_}")
print(f"Лучшая точность при кросс валидации: {grid_search.best_score_:.2f}")

Гиперпараметры модели с лучшим результатом: {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.01}
Лучшая точность при кросс валидации: 0.70


## Тест наилучшей модели

In [11]:
test_accuracy = grid_search.score(X_test_scaled, y_test) # Подставьте данные тестовой выборки
print(f"Точность на тестовой выборке: {test_accuracy:.2f}")

Точность на тестовой выборке: 0.74


In [12]:
# Если понадобится сохранить лучший вариант, то он находится в атрибуте best_estimator_
# Все данные по кросс валидации находятся в атрибуте cv_results_

best_model = grid_search.best_estimator_

**Что такое эпоха (epoch) при обучении модели?**

Ваш ответ: один полный проход алгоритма обучения по всему обучающему набору данных.

**Сколько эпох задается изначально при создании MLPClassifier?**

Ваш ответ: По умолчанию при создании объекта MLPClassifier в библиотеке sklearn количество эпох (заданное как max_iter) равно 200. Это означает, что если не указать значение max_iter явно, модель будет обучаться на протяжении 200 эпох, если не достигнет критерия сходимости раньше.

**Для чего требуется регуляризация L2 или L1?**

Ваш ответ: Регуляризация L2 и L1 используются для предотвращения переобучения модели за счет добавления штрафа к величине весов. Эти методы помогают уменьшить сложность модели и снижать риск переобучения, заставляя веса быть более малыми или даже нулевыми.

**Сколько параметров в вашей лучшей модели?**

Ваш ответ: имеет один скрытый слой с 100 нейронами и использует функцию активации ReLU.